In [13]:
import numpy as np
import open3d as o3d
from superprimitive_fusion.scanner import capture_spherical_scans
from superprimitive_fusion.utils import bake_uv_to_vertex_colours, distinct_colours
from superprimitive_fusion.mesh_fusion import fuse_meshes

In [14]:
mesh = o3d.io.read_triangle_mesh("../data/power-drill/textured.obj", enable_post_processing=True)

bake_uv_to_vertex_colours(mesh)

mesh.compute_vertex_normals()

bb = mesh.get_minimal_oriented_bounding_box()
scale = np.mean(bb.get_max_bound())

In [15]:
scans = capture_spherical_scans(
    mesh=mesh,
    num_views=6,
    radius=0.3,
    width_px=360,
    height_px=240,
    fov=70,
    dropout_rate=0,
    depth_error_std=0.0,
    translation_error_std=0,
    rotation_error_std_degs=0,
    dist_thresh=0.25,
    sampler="fibonacci",
)
meshes = [scan['mesh'] for scan in scans]

o3d.visualization.draw_geometries(
    meshes,
    window_name="Virtual scan",
    front=[0.3, 1, 0],
    lookat=[0, 0, 0],
    up=[0, 0, 1],
    zoom=0.7,
)

In [16]:
for i in range(len(meshes)):
    o3d.visualization.draw_geometries(
        [meshes[i]],
        window_name="Virtual scan",
        front=[0.3, 1, 0],
        lookat=[0, 0, 0],
        up=[0, 0, 1],
        zoom=0.7,
    )

In [17]:
j = 6
fused_mesh = meshes[0]
for i in range(1,j):
    fused_mesh = fuse_meshes(
        fused_mesh,
        meshes[i],
        h_alpha=5,
        trilat_iters=2,
        shift_all=False,
        fill_holes=True,
    )

o3d.visualization.draw_geometries(
    [fused_mesh],
    window_name="Virtual scan",
    front=[0.3, 1, 0],
    lookat=[0, 0, 0],
    up=[0, 0, 1],
    zoom=0.7,
)

In [18]:
import copy

mesh = copy.deepcopy(fused_mesh)

out = mesh.cluster_connected_triangles()

clust_ids = np.asarray(out[0])
unique_clusters = np.unique(clust_ids)
print(f'There are {len(unique_clusters)} clusters of connected components')

colours = distinct_colours(len(unique_clusters))
colours = (colours * 255).astype(np.uint8)

components = []
for i in unique_clusters:
    component = copy.deepcopy(mesh)
    tris = np.asarray(component.triangles)
    tris_clust = tris[clust_ids==i]
    component.triangles = o3d.utility.Vector3iVector(tris_clust)
    component.paint_uniform_color(colours[i].astype(float) / 255)
    components.append(component)

o3d.visualization.draw_geometries(
    components
)

There are 22 clusters of connected components
